In [1]:
# Mount Google Drive to access files stored in the user's Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install the Gradio library for creating the web interface
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.0 MB/s eta 0:00:00


In [3]:
# Install all required dependencies from the specified requirements file
!pip install -r "/content/drive/MyDrive/Langchain/Projects/Project 1/requirements.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.5 MB/s eta 

In [4]:
# Import the Gemini API wrapper from LangChain
from langchain_google_genai import ChatGoogleGenerativeAI

# Import the prompt templating utility
from langchain_core.prompts import PromptTemplate

# Import parser to extract string output from the model
from langchain_core.output_parsers import StrOutputParser

# Import message types for structured conversations
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

# Load environment variables from a .env file (e.g., API keys)
from dotenv import load_dotenv

# Import Gradio for creating the frontend UI
import gradio as gr


In [5]:
# Load environment variables from the specified .env file path
load_dotenv(dotenv_path="/content/drive/MyDrive/Langchain/Projects/Project 1/.env")

True

In [6]:
# Initialize the Gemini model with the 'gemini-2.0-flash-lite' variant
model = ChatGoogleGenerativeAI(model='gemini-2.0-flash-lite')

# Parser to extract and format plain text output from the AI response
parser = StrOutputParser()

In [7]:
# Define the system's default behavior/prompt for the assistant
initial_system_message = SystemMessage(
    content="You are a helpful assistant. Always reply with only one concise line, no matter what."
)

In [8]:
# Initialize a list to track the ongoing chat history
chat_history = [initial_system_message]

In [9]:
# Function to handle user input and generate bot response
def chat_with_bot(user_input, history=[]):
    global chat_history  # Use the global chat history list

    chat_history.append(HumanMessage(content=user_input))  # Add user's message to history

    result = model.invoke(chat_history)  # Get response from the Gemini model
    chat_history.append(result)  # Add AI's response to history

    history.append((user_input, result.content))  # Update the Gradio chat history
    return history, history  # Return updated history for display and state

In [10]:
# Function to reset the chat (clears conversation)
def reset_chat():
    global chat_history
    chat_history = [initial_system_message]  # Reset to initial system message
    return [], []  # Return empty chat and state for Gradio

In [13]:
# Define the Gradio UI layout and behavior
with gr.Blocks() as demo:
    gr.Markdown("# Chatbot (LangChain + Gradio)")  # Title

    chatbot = gr.Chatbot()  # Visual chat window
    user_input = gr.Textbox(label="You", placeholder="Type your message...")  # Input field
    send_btn = gr.Button("Send")  # Button to submit messages
    clear_btn = gr.Button("Reset Chat")  # Button to reset the chat

    state = gr.State([])  # Internal state to store chat history in Gradio

    # On clicking 'Send', call chat_with_bot, pass user input and state, and update chatbot display
    send_btn.click(fn=chat_with_bot, inputs=[user_input, state], outputs=[chatbot, state])

    # On clicking 'Reset Chat', call reset_chat and clear UI + state
    clear_btn.click(fn=reset_chat, outputs=[chatbot, state])

<ipython-input-13-53d741f96182>:5: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


In [14]:
# Launch the Gradio app
demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7a484393dd6a202aa4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
